# Excercise 2 - Metadata Exploration

Description: 

In this exercise we will access metadata about each feature layer: the feature class name, geometry type, and projected coordinate system. 



## Set up

In [1]:
# import python packages

import sys
import glob2 as glob
from osgeo import ogr,gdal
import fiona
import geopandas as gpd
# conda env packages: glob2, gdal, geopandas, fiona, juptyer
# create conda env from DEV_py_demo yml file


In [3]:
# list any files or directories we will use

# data_dir = 'C://Users/kerrie/Documents/ArcGIS/Projects/DEV_python/export_gdb_data/' # update to wherever you exported your data
# workspace_dir = 'C://Users/kerrie/Documents/01_LocalCode/repos/MSU_example_compute/python/DEV_python_canvas_course/' # update to wherever you saved your jupyter notebook
gdb_file='C://Users/kerrie/Documents/ArcGIS/Projects/DEV_python/Ex_02/Ex2_GIS_data/Ex2_GISdata.gdb'

## Metadata Exploration

There are multiple ways to read data out of an ESRI geodatabase using free open source python packages. The newer and more intuitive way to do this is with the fiona and geopandas packages, which use gdal under the hood. The older method that has more functionality for geospatial data but is also a bit more convoluted is to use the osgeo and gdal packages. Here, we demonstrate the same task twice. Once using geopandas+fiona and once with osgeo+gdal. There may be other python packages that can accomplish this task but we won't cover them here.

The task is to open an ESRI geodatabase, get all the feature class/layer names in the file, and get the geometry type and projected coordinate system for each layer.


#### Using geopandas + fiona

In [4]:
# get layer names, geometry types, projected coordinate system names

for layer_name in fiona.listlayers(gdb_file):
    # using fiona above to loop thru the list of layer names

    # read a layer, .read_file can read shp and gdb
    gdf=gpd.read_file(gdb_file,layer=layer_name) 

    # .geom_type returns the geometry for every data record in the layer
    # .unique() returns a list of only 1 type if all records have the same geometry
    geom_type=gdf.geom_type.unique()  

    # .crs returns spatial reference information, get the name of the projected coord system
    sr_name=gdf.crs.name

    print("Feature class name: {} \nGeometry Type: {}\nProjected Coordinate System: {}"
        "\n".format(layer_name, geom_type, sr_name))

Feature class name: towns_105 
Geometry Type: ['Point']
Projected Coordinate System: mstm

Feature class name: desig_hwys_105 
Geometry Type: ['MultiLineString']
Projected Coordinate System: NAD83 / Mississippi TM

Feature class name: pop_pl_105 
Geometry Type: ['MultiPolygon']
Projected Coordinate System: mstm

Feature class name: co_105 
Geometry Type: ['MultiPolygon']
Projected Coordinate System: mstm

Feature class name: VolusiaCounty 
Geometry Type: ['MultiPolygon']
Projected Coordinate System: NAD83_HARN_Florida_East_ftUS

Feature class name: Highways 
Geometry Type: ['MultiLineString']
Projected Coordinate System: NAD83_HARN_Florida_East_ftUS

Feature class name: Schools 
Geometry Type: ['Point']
Projected Coordinate System: NAD83_HARN_Florida_East_ftUS



#### Using osgeo + gdal

In [5]:
# these packages won't always throw errors if something goes wrong
# we have to turn on exceptions if we want this behavior and do a little extra error catching
ogr.UseExceptions()

# get the appropriate driver for reading gdb
try: driver_gdb=ogr.GetDriverByName("OpenFileGDB")
except Exception as e: sys.exit(e)

# open the gdb as read only (0)
try: gdb=driver_gdb.Open(gdb_file,0)
except Exception as e: sys.exit(e)

In [6]:
# get layer names, geometry types, projected coordinate system names
for ilayer in range(gdb.GetLayerCount()):
    # using gdal above to loop thru a list of layer indexes

    # read a layer
    fclass=gdb.GetLayerByIndex(ilayer)
    
    # get the layer name
    layer_name=fclass.GetName()

    # get geometry type
    geom_type=ogr.GeometryTypeToName(fclass.GetGeomType())

    # .GetSpatialRef() returns spatial reference information, get the name of the projected coord system
    sr_name=fclass.GetSpatialRef().GetAttrValue('PROJCS')
    
    print("Feature class name: {} \nGeometry Type: {}\nProjected Coordinate System: {}"
        "\n".format(layer_name,geom_type, sr_name))    

Feature class name: towns_105 
Geometry Type: Point
Projected Coordinate System: mstm

Feature class name: desig_hwys_105 
Geometry Type: Multi Line String
Projected Coordinate System: NAD83 / Mississippi TM

Feature class name: pop_pl_105 
Geometry Type: Multi Polygon
Projected Coordinate System: mstm

Feature class name: co_105 
Geometry Type: Multi Polygon
Projected Coordinate System: mstm

Feature class name: VolusiaCounty 
Geometry Type: Multi Polygon
Projected Coordinate System: NAD83_HARN_Florida_East_ftUS

Feature class name: Highways 
Geometry Type: Multi Line String
Projected Coordinate System: NAD83_HARN_Florida_East_ftUS

Feature class name: Schools 
Geometry Type: Point
Projected Coordinate System: NAD83_HARN_Florida_East_ftUS

